# Separate SWR from wake versus sleep time

We ask whether L1 activity around SWR is different in wake and sleep stages

In [ ]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import xarray as xr
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
%matplotlib widget
import scipy

from ephyviewer import mkQApp, MainViewer, TraceViewer

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)



In [ ]:
folder_base = Path('/Users/ahay/Documents/DATA/L1imaging/Gaelle/Baseline_recording/RedLinesOK/session1/')

folder_Ephys = folder_base / f'OpenEphys/'
folder_Minis = folder_base / f'V4_Miniscope/minian/'

filenameE = folder_Ephys / f'SWRproperties.csv'
SWRprop = pd.read_csv(filenameE)
filenameW0 = folder_Ephys / f'LFPwake0.npy'
Wake = np.load(filenameW0, mmap_mode= 'r')
Wake0 = Wake[:,1]
filenameB = folder_base / f'SynchroFile.xlsx'
stamps = pd.read_excel(filenameB)
filenameU = folder_Minis / f'UniTtoKeep.xlsx'
UnitToKeep = pd.read_excel(filenameU)
UnitToKeepL = UnitToKeep[0].to_list()
minian_ds = open_minian(folder_Minis)


for file_path in folder_base.glob('**/*timeStamps.csv'):
    subfolder = file_path.parents[0].stem
    if subfolder == 'V4_Miniscope':
        stamps_miniscope = pd.read_csv(file_path)

Acq_freq = (list(stamps[0])[2])

TodropFile = folder_Minis / f'TodropFile.json'
with open(TodropFile, 'r') as f:
    unit_to_drop = json.load(f)
TodropFile = folder_Minis / f'AllunitsFile.json'
with open(TodropFile, 'r') as f:
    All_units = json.load(f)

UnitToKeepLsel = []
UnitToKeepInd = [e for e, elem in enumerate(All_units) if elem in UnitToKeepL]
UnitToKeepElem = [elem for e, elem in enumerate(All_units) if elem in UnitToKeepL]
print(UnitToKeepElem)

Force to recreate a boolean because of compatibility issues between versions of pandas (original boolean created with version of Pandas newer than the one required to run Minian)

In [ ]:
Bool = Wake0.copy()
for i in range(len(Wake0)-1):
    if (Wake0[i]==0 and Wake0[i + 1]==0):
        Bool[i] = 0
    else:
        Bool[i] = 1

In [ ]:
SWRprop[["peak time", "start time", "end time"]] = SWRprop[["peak time", "start time", "end time"]]-(list(stamps[0])[0]*1000)
C = minian_ds['S']
A = minian_ds['A']
B = A['unit_id']

copyB = list(B.copy())
for i in range(len(unit_to_drop)):
    elem = unit_to_drop[i]
    copyB.remove(elem)
unit_to_keep = copyB

C = C.loc[unit_to_keep,:]
nb_unit = C.shape[0]
rec_dur = C.shape[1]

In [ ]:
counterAwakeSWR = 0
counterSleepSWR = 0

ScoringVectorLength = len(Bool)
WakeStatus = np.zeros((ScoringVectorLength))
WakeStatus = Bool

SWRawake = list()
SWRsleep = list()

serSWR = pd.Series(SWRprop["peak time"]).to_list()
nb_SWR = SWRprop.shape[0]
for Pswr in range(nb_SWR):
    time = round(serSWR[Pswr])
    if (WakeStatus[time] == 1): # SWR in sleep
        counterSleepSWR = counterSleepSWR + 1
        SWRsleep.append(Pswr)
    else:
        SWRawake.append(Pswr)
        counterAwakeSWR = counterAwakeSWR + 1

print(counterSleepSWR, counterAwakeSWR, nb_SWR)


In [ ]:
from bisect import bisect_left

def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before


def take_closest3(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        dummy = myList.index(after)
        return dummy
    else:
        dummy = myList.index(before)
        return dummy


In [ ]:
Cseries = C.to_series()
units = range(len(UnitToKeepInd))
maxi = len(UnitToKeepInd)
duration = 1
numbframes = int(duration*2*Acq_freq)
threshold = 0


data = {
    'SleepAll': np.zeros(maxi),
    'SleepBefore': np.zeros(maxi),
    'SleepAfter' : np.zeros(maxi),
    'WakeAll' : np.zeros(maxi),
    'WakeBefore' : np.zeros(maxi),
    'WakeAfter' : np.zeros(maxi),
    'UnitId' : np.zeros(maxi)
}
Activity_Per_Neurons = pd.DataFrame(data, columns=['SleepAll', 'SleepBefore', 'SleepAfter', 'WakeAll', 'WakeBefore', 'WakeAfter', 'UnitId'])

Activity_Per_Neurons['UnitId'] = UnitToKeepElem

ActivityCa_SWR = pd.Series(np.zeros(numbframes))
time = range(numbframes)
Half = int(ActivityCa_SWR.shape[0]/2)
TimeStamps_miniscope = list(stamps_miniscope["Time Stamp (ms)"])
plt.close()

fig, axs = plt.subplots(nrows=maxi, ncols=1)


AVGActivityCa_SleepSWR = pd.Series(np.zeros(numbframes))
nAVGActivityCa_SleepSWR = pd.Series(np.zeros(numbframes))
for unit in units:
    ActivityCa_SleepSWR = pd.Series(np.zeros(numbframes))
    AVGlocActivityCa_SleepSWR = pd.Series(np.zeros(numbframes))
    lCseries = C.isel(unit_id=unit).values
    for Pspin in SWRsleep:
        ser = pd.Series(SWRprop["start time"])
        Frame_SWR_start = take_closest(TimeStamps_miniscope, ser[Pspin])
        index = TimeStamps_miniscope.index(Frame_SWR_start)
        trace = lCseries[index-Half:index+Half]
        if len(trace)<len(time):
            print("pre, unit = ", unit, "SWR = ", Pspin)
        else:
            ActivityCa_SleepSWR = ActivityCa_SleepSWR+list(lCseries[index-Half:index+Half])# move by half for CTRL
    if max(ActivityCa_SleepSWR)>(len(SWRsleep)*threshold):
        nActivityCa_SleepSWR = ActivityCa_SleepSWR/max(ActivityCa_SleepSWR)
    elif sum(trace)>threshold:
        nActivityCa_SleepSWR = 0#ActivityCa_PFCspin
    
    AVGActivityCa_SleepSWR = AVGActivityCa_SleepSWR + ActivityCa_SleepSWR    
    AVGlocActivityCa_SleepSWR = AVGlocActivityCa_SleepSWR + ActivityCa_SleepSWR    
    nAVGActivityCa_SleepSWR = nAVGActivityCa_SleepSWR + nActivityCa_SleepSWR 
    ax = axs[unit]
    ax.plot(time, AVGlocActivityCa_SleepSWR/counterSleepSWR, color='blue') 
    Activity_Per_Neurons['SleepAll'][unit] = sum(AVGlocActivityCa_SleepSWR[0:2*Half])
    Activity_Per_Neurons['SleepBefore'][unit] = sum(AVGlocActivityCa_SleepSWR[0:Half])
    Activity_Per_Neurons['SleepAfter'][unit] = sum(AVGlocActivityCa_SleepSWR[Half:2*Half])
    
if max(AVGActivityCa_SleepSWR)>(threshold):
    nAVGActivityCa_SleepSWR = AVGActivityCa_SleepSWR/max(AVGActivityCa_SleepSWR)
else:
    nAVGActivityCa_SleepSWR = AVGActivityCa_SleepSWR


prCactBefore = sum(AVGActivityCa_SleepSWR[0:Half])
prCactAfter = sum(AVGActivityCa_SleepSWR[Half:2*Half])
nprCactBefore = sum(nAVGActivityCa_SleepSWR[0:Half])
nprCactAfter = sum(nAVGActivityCa_SleepSWR[Half:2*Half])


AVGActivityCa_WakeSWR = pd.Series(np.zeros(numbframes))
nAVGActivityCa_WakeSWR = pd.Series(np.zeros(numbframes))
for unit in units:
    AVGlocActivityCa_WakeSWR = pd.Series(np.zeros(numbframes))
    ActivityCa_WakeSWR = pd.Series(np.zeros(numbframes))
    lCseries = np.array(Cseries)[(unit)*rec_dur:(unit+1)*rec_dur]
    for Pspin in SWRawake:
        ser = pd.Series(SWRprop["start time"])
        Frame_SWR_start = take_closest(TimeStamps_miniscope, ser[Pspin])
        index = TimeStamps_miniscope.index(Frame_SWR_start)
        trace = list(lCseries[index-Half:index+Half])
        if len(trace)<len(time):
            print("post, unit = ", unit, "SWR = ", Pspin)
        else:
            ActivityCa_WakeSWR = ActivityCa_WakeSWR+list(lCseries[index-Half:index+Half])# move by half for CTRL
    if max(ActivityCa_WakeSWR)>(len(SWRawake)*threshold):
        nActivityCa_WakeSWR = ActivityCa_WakeSWR/max(ActivityCa_WakeSWR)
    elif sum(trace)>threshold:
        nActivityCa_WakeSWR = 0
    AVGActivityCa_WakeSWR = AVGActivityCa_WakeSWR + ActivityCa_WakeSWR
    AVGlocActivityCa_WakeSWR = AVGlocActivityCa_WakeSWR + ActivityCa_WakeSWR   
    nAVGActivityCa_WakeSWR = nAVGActivityCa_WakeSWR + nActivityCa_WakeSWR 
    ax = axs[unit]
    ax.plot(time, AVGlocActivityCa_WakeSWR/counterAwakeSWR, color='green') 
    Activity_Per_Neurons['WakeAll'][unit] = sum(AVGlocActivityCa_WakeSWR[0:2*Half])
    Activity_Per_Neurons['WakeBefore'][unit] = sum(AVGlocActivityCa_WakeSWR[0:Half])
    Activity_Per_Neurons['WakeAfter'][unit] = sum(AVGlocActivityCa_WakeSWR[Half:2*Half])

if max(AVGActivityCa_WakeSWR)>(threshold):
    nAVGActivityCa_WakeSWR = AVGActivityCa_WakeSWR/max(AVGActivityCa_WakeSWR)
else:
    nAVGActivityCa_WakeSWR = AVGActivityCa_WakeSWR


poCactBefore = sum(AVGActivityCa_WakeSWR[0:Half])
poCactAfter = sum(AVGActivityCa_WakeSWR[Half:2*Half])
npoCactBefore = sum(nAVGActivityCa_WakeSWR[0:Half])
npoCactAfter = sum(nAVGActivityCa_WakeSWR[Half:2*Half])

